In [ ]:
### Fase 1:

In [1]:
import random
import math

class Node:
    """Represents a node (location) in the rescue graph."""
    def __init__(self, name, x, y):
        self.name = name
        self.x = x  # X-coordinate
        self.y = y  # Y-coordinate
        self.tasks = []  # Tasks at this node

    def add_task(self, task):
        """Add a task to this node."""
        self.tasks.append(task)

    def distance_to(self, other_node):
        """Calculate the distance to another node."""
        return math.sqrt((self.x - other_node.x)**2 + (self.y - other_node.y)**2)

class Task:
    """Represents a task at a node."""
    def __init__(self, task_id, visual_search, material_transport, medical_assistance, personnel_transfer, optimal_time, effective_time, ineffective_time):
        self.task_id = task_id  # Unique ID for the task
        self.visual_search = visual_search  # True/False if visual search is needed
        self.material_transport = material_transport  # Material weight required (kg)
        self.medical_assistance = medical_assistance  # Number of patients needing assistance
        self.personnel_transfer = personnel_transfer  # Number of personnel to transfer
        self.optimal_time = optimal_time  # Optimal rescue time for maximum benefit
        self.effective_time = effective_time  # Effective rescue time for reduced benefit
        self.ineffective_time = ineffective_time  # Time after which no benefit is awarded
        self.is_completed = False  # Track if task is completed
        self.completion_time = None  # Time when the task was completed

    def calculate_benefit(self, completion_time):
        """Calculate the benefit based on the completion time."""
        if completion_time <= self.optimal_time:
            return 1  # Maximum benefit
        elif completion_time <= self.effective_time:
            # Reduced benefit
            return 0.5 * ((self.effective_time - completion_time) / (self.effective_time - self.optimal_time))
        else:
            return 0  # No benefit


class Helicopter:
    """Represents a helicopter in the rescue fleet."""
    def __init__(self, helicopter_id, hovering, visual_search_capability, load_capacity, medical_capacity, passenger_limit, range_km, speed_kmh, position):
        self.helicopter_id = helicopter_id  # Unique ID
        self.hovering = hovering  # Whether it can hover (True/False)
        self.visual_search_capability = visual_search_capability  # True/False if visual search is possible
        self.load_capacity = load_capacity  # Maximum load in kg
        self.medical_capacity = medical_capacity  # Maximum medical assistance capacity
        self.passenger_limit = passenger_limit  # Maximum passengers
        self.range_km = range_km  # Maximum range in km
        self.speed_kmh = speed_kmh  # Speed in km/h
        self.position = position  # Current node (Node object)

    def can_complete_task(self, task):
        """Check if the helicopter can complete a given task."""
        return (
            (not task.visual_search or self.visual_search_capability) and
            self.load_capacity >= task.material_transport and
            self.medical_capacity >= task.medical_assistance and
            self.passenger_limit >= task.personnel_transfer
        )

    def complete_task(self, task, completion_time):
        """Complete a task if resources are sufficient and calculate benefit."""
        if self.can_complete_task(task):
            self.load_capacity -= task.material_transport
            self.medical_capacity -= task.medical_assistance
            self.passenger_limit -= task.personnel_transfer
            task.is_completed = True
            task.completion_time = completion_time
            return task.calculate_benefit(completion_time)
        return 0

def initialize_graph():
    """Initialize the rescue graph with nodes, tasks, and helicopters."""
    # Define nodes
    central_node = Node("Central Base", 0, 0)  # Centralized base
    nodes = [
        central_node,
        Node("Node A", 10, 5),
        Node("Node B", 20, 15),
        Node("Node C", 30, 10),
        Node("Node D", 40, 20)
    ]

    # Add tasks to nodes
    tasks = [
        Task(task_id=1, visual_search=True, material_transport=200, medical_assistance=2, personnel_transfer=1, optimal_time=20, effective_time=40, ineffective_time=60),
        Task(task_id=2, visual_search=False, material_transport=300, medical_assistance=1, personnel_transfer=2, optimal_time=15, effective_time=30, ineffective_time=50),
        Task(task_id=3, visual_search=True, material_transport=100, medical_assistance=3, personnel_transfer=0, optimal_time=10, effective_time=25, ineffective_time=40),
        Task(task_id=4, visual_search=False, material_transport=400, medical_assistance=0, personnel_transfer=3, optimal_time=30, effective_time=50, ineffective_time=70),
        Task(task_id=5, visual_search=True, material_transport=150, medical_assistance=1, personnel_transfer=1, optimal_time=25, effective_time=45, ineffective_time=60),
        Task(task_id=6, visual_search=True, material_transport=100, medical_assistance=3, personnel_transfer=0, optimal_time=10, effective_time=25, ineffective_time=120),
        Task(task_id=7, visual_search=False, material_transport=400, medical_assistance=0, personnel_transfer=3, optimal_time=30, effective_time=50, ineffective_time=90),
        Task(task_id=8, visual_search=True, material_transport=150, medical_assistance=1, personnel_transfer=1, optimal_time=25, effective_time=45, ineffective_time=100),
    ]
    nodes[1].add_task(tasks[0])
    nodes[1].add_task(tasks[1])
    nodes[2].add_task(tasks[2])
    nodes[2].add_task(tasks[7])
    nodes[2].add_task(tasks[6])
    nodes[3].add_task(tasks[4])
    nodes[3].add_task(tasks[5])
    nodes[4].add_task(tasks[3])

    # Define helicopters stationed at the central base
    helicopters = [
        Helicopter(1, hovering=True, visual_search_capability=True, load_capacity=500, medical_capacity=3, passenger_limit=3, range_km=500, speed_kmh=200, position=central_node),
        Helicopter(2, hovering=True, visual_search_capability=False, load_capacity=500, medical_capacity=3, passenger_limit=3, range_km=500, speed_kmh=200, position=central_node),
        Helicopter(3, hovering=True, visual_search_capability=True, load_capacity=500, medical_capacity=3, passenger_limit=3, range_km=500, speed_kmh=200, position=central_node),
        Helicopter(4, hovering=True, visual_search_capability=False, load_capacity=500, medical_capacity=3, passenger_limit=3, range_km=500, speed_kmh=200, position=central_node),
    ]

    return nodes, helicopters



# Test initialization
nodes, helicopters = initialize_graph()

# Example benefit calculation
task = nodes[1].tasks[0]  # Select a task from Node A
helicopter = helicopters[0]
completion_time = 20  # Simulated completion time
benefit = helicopter.complete_task(task, completion_time)
print(f"Benefit for completing task {task.task_id} at time {completion_time}: {benefit}")

Benefit for completing task 1 at time 20: 1


In [ ]:
## Fase 2:

In [2]:
def assign_task(helicopter, nodes, current_time):
    """
    Assign a task to the helicopter based on its capacity and task priority.
    Tasks are evaluated for maximum benefit.
    """
    best_task = None
    best_task_node = None
    max_benefit = 0

    for node in nodes:
        for task in node.tasks:
            if not task.is_completed:
                # Calculate travel time to the task
                travel_time = helicopter.position.distance_to(node)
                completion_time = current_time + travel_time

                # Check if the helicopter can complete the task
                if helicopter.can_complete_task(task):
                    benefit = task.calculate_benefit(completion_time)
                    if benefit > max_benefit:
                        max_benefit = benefit
                        best_task = task
                        best_task_node = node

    if best_task:
        # Move the helicopter to the task's node
        travel_time = helicopter.position.distance_to(best_task_node)
        helicopter.position = best_task_node
        print(
            f"Helicopter {helicopter.helicopter_id} assigned to Task {best_task.task_id} at Node {best_task_node.name}. Travel time: {travel_time}."
        )
        return best_task, travel_time
    else:
        print(f"Helicopter {helicopter.helicopter_id} has no suitable tasks.")
        return None, 0


def simulate_rescue(nodes, helicopters, total_time):
    """
    Simulate the rescue operation over a given total time.
    """
    current_time = 0
    total_benefit = 0

    while current_time < total_time:
        for helicopter in helicopters:
            # Assign a task to the helicopter
            task, travel_time = assign_task(helicopter, nodes, current_time)

            if task:
                # Update the time to reflect travel and task completion
                current_time += travel_time
                task_completion_time = current_time

                # Complete the task and calculate its benefit
                benefit = helicopter.complete_task(task, task_completion_time)
                total_benefit += benefit

                print(
                    f"Task {task.task_id} completed by Helicopter {helicopter.helicopter_id} at time {task_completion_time}. Benefit: {benefit}"
                )

        # Increment the simulation time (if tasks were idle)
        current_time += 1

    print(f"\nSimulation completed. Total benefit: {total_benefit}")


In [3]:
# Initialize the rescue graph
nodes, helicopters = initialize_graph()

# Run the simulation for a total of 100 time units
simulate_rescue(nodes, helicopters, total_time=100)


Helicopter 1 assigned to Task 1 at Node Node A. Travel time: 11.180339887498949.
Task 1 completed by Helicopter 1 at time 11.180339887498949. Benefit: 1
Helicopter 2 assigned to Task 7 at Node Node B. Travel time: 25.0.
Task 7 completed by Helicopter 2 at time 36.180339887498945. Benefit: 0.3454915028125264
Helicopter 3 has no suitable tasks.
Helicopter 4 has no suitable tasks.
Helicopter 1 has no suitable tasks.
Helicopter 2 has no suitable tasks.
Helicopter 3 has no suitable tasks.
Helicopter 4 has no suitable tasks.
Helicopter 1 has no suitable tasks.
Helicopter 2 has no suitable tasks.
Helicopter 3 has no suitable tasks.
Helicopter 4 has no suitable tasks.
Helicopter 1 has no suitable tasks.
Helicopter 2 has no suitable tasks.
Helicopter 3 has no suitable tasks.
Helicopter 4 has no suitable tasks.
Helicopter 1 has no suitable tasks.
Helicopter 2 has no suitable tasks.
Helicopter 3 has no suitable tasks.
Helicopter 4 has no suitable tasks.
Helicopter 1 has no suitable tasks.
Helicop